In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from glob import glob

In [4]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 12.2 MB/s eta 0:00:00


In [23]:
import os
import numpy as np
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

In [24]:
images=glob('/content/drive/MyDrive/LandSat8/*T*[1-2]_SR_B*[1-7]*.TIF')
images

['/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B1.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B2.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B3.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B5.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B4.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B6.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240409_20240418_02_T1_SR_B7.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240612_20240628_02_T2_SR_B1.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240612_20240628_02_T2_SR_B2.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240612_20240628_02_T2_SR_B3.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP_146051_20240612_20240628_02_T2_SR_B4.TIF',
 '/content/drive/MyDrive/LandSat8/LC08_L2SP

In [ ]:

for image in images:
  src = rasterio.open(image)
  show(src)


In [42]:
def calculate_ndvi(nir_band, red_band):
    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (nir_band - red_band) / (nir_band + red_band)
        ndvi[np.isnan(ndvi)] = 0
        ndvi[np.isinf(ndvi)] = 0
    return ndvi

In [45]:
def create_cloud_mask(ndvi, threshold=0.3):
    cloud_mask = ndvi < threshold
    return cloud_mask

In [46]:
scene_dates = ['20240612_20240628', '20240409_20240418']

In [ ]:
scene_dates = ['20240612_20240628', '20240409_20240418']
for scene_date in scene_dates:
    # Select the files for the current scene
    scene_files = [os.path.join(folder_path, f) for f in images if scene_date in f]

    # Dictionary to store the bands
    bands = {}

    # Load the bands
    for file in scene_files:
        with rasterio.open(file) as src:
            band_name = os.path.basename(file).split('_')[-1].split('.')[0]
            bands[band_name] = src.read(1)
            profile = src.profile

    # Assuming the bands are labeled as B1, B2, ..., B7
    # Select NIR (B5) and Red (B4) bands
    nir_band = bands['B5']
    red_band = bands['B4']

    # Calculate NDVI
    ndvi = calculate_ndvi(nir_band, red_band)

    # Plot NDVI Histogram to determine threshold
    plt.figure(figsize=(10, 6))
    plt.hist(ndvi.ravel(), bins=50, color='green')
    plt.title('NDVI Histogram')
    plt.xlabel('NDVI Values')
    plt.ylabel('Frequency')
    plt.show()

    # Create cloud mask
    cloud_mask = create_cloud_mask(ndvi, threshold=0.1)

    # Plot the results
    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    show(red_band, ax=ax[0], title='Red Band')
    show(nir_band, ax=ax[1], title='NIR Band')
    show(cloud_mask, ax=ax[2], title='Cloud Mask', cmap='gray')
    plt.show()

    # Save the cloud mask as a new file
    mask_filename = f'/content/drive/MyDrive/LandSat8/cloud_mask_{scene_date}.tif'
    with rasterio.open(
        mask_filename,
        'w',
        driver='GTiff',
        height=cloud_mask.shape[0],
        width=cloud_mask.shape[1],
        count=1,
        dtype=rasterio.uint8,
        crs=profile['crs'],
        transform=profile['transform'],
    ) as dst:
        dst.write(cloud_mask.astype(rasterio.uint8), 1)

print("Cloud masking completed.")

In [ ]:
bands

for bands, profile in images1:
    print(f"Number of bands: {len(bands)}")